In [6]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
# from src.load_transform_data import get_new_ethereum_ohlc, get_new_ethereum_ohlc_2
from src.hopsworks_connections import pull_data, upload_data, pull_model
import pandas_ta as ta
import numpy as np
import requests
from datetime import datetime, timedelta
import time

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


## Earlier, I used to scrape data from coinlore.com, but I realized they update it several hours late every day, so now I use coinGecko API instead. This was my code for that...

### Download the last 30 days of Ethereum OHLC data from coinlore.com

In [2]:
##### This is the scraping function that is used in github actions - it wont work locally unless you have chromeDriver installed #####
# But if you want ChromeDriver locally, you can do this (Mac):
# 1. brew install chromedriver
# 2. xattr -d com.apple.quarantine $(which chromedriver)
# 3. which chromedriver
# 4. the above line give the path to chromedriver. Now you can go into src/load_transfrom_data.py, and change the path in the get_new_ethereum_ohlc() function.  
# new_eth_ohlc = get_new_ethereum_ohlc()

##### Use this function when you running locally. This one doesn't require chromeDriver
# new_eth_ohlc = get_new_ethereum_ohlc_2()

# new_eth_ohlc.head()

### clean the new raw ethereum data so hopsworks accepts it

In [3]:
# def convert_value(value):
#     """
#     Converts a string value to a float. Removes $ signs, and converts
#     billion (bn), million (m), and thousand (K) values to their numeric equivalents.
#     """
#     value = value.replace('$', '')  # Remove $ sign to simplify processing
#     if value[-1].lower() == 'm':
#         return float(value[:-1]) * 1_000_000
#     elif value[-1].lower() == 'b':
#         return float(value[:-1]) * 1_000_000_000
#     elif value[-1].lower() == 'k':
#         return float(value[:-1]) * 1_000
#     elif value[-2:].lower() == 'bn':  # Handle 'bn' for billions
#         return float(value[:-2]) * 1_000_000_000
#     else:
#         return float(value)

# new_eth_ohlc.columns = [col.lower() for col in new_eth_ohlc.columns]
# new_eth_ohlc.rename(columns={'volume(eth)': 'volume_eth', 'market cap': 'market_cap'}, inplace=True)

# for col in ['open', 'high', 'low', 'close', 'volume', 'market_cap']:
#     new_eth_ohlc[col] = new_eth_ohlc[col].apply(convert_value)

# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date'])

# new_eth_ohlc.head()

### Combine this latest raw Etherum data with existing raw Ethereum data in hopsworks, and save it back into hopsworks

In [4]:
# raw_eth_ohlc = pull_data('raw_ethereum_ohlc', 1, 'raw_ethereum_ohlc_view', 1)
# raw_eth_ohlc

In [5]:
# # Hopsowrks returns fucked date column object, with weird time zone formats, we need to standardize first
# raw_eth_ohlc['date'] = pd.to_datetime(raw_eth_ohlc['date']).dt.tz_localize(None)
# new_eth_ohlc['date'] = pd.to_datetime(new_eth_ohlc['date']).dt.tz_localize(None)

# combined_raw_eth_ohlc = pd.concat([new_eth_ohlc, raw_eth_ohlc], ignore_index=True)
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.drop_duplicates(subset='date', keep='last')
# combined_raw_eth_ohlc = combined_raw_eth_ohlc.sort_values(by='date')
# combined_raw_eth_ohlc

In [6]:
# upload_data(combined_raw_eth_ohlc, 'raw_ethereum_ohlc', 1, 'Raw ethereum OHLC (open, high, low, close) data from coinlore.com')

## Get recent data using CoinGecko API

In [2]:
datetime.now()

datetime.datetime(2024, 5, 14, 19, 12, 55, 400295)

In [3]:
def fetch_recent_data(crypto):
    # 20:00, or 8pm in EST is 12:00 in UTC, which is the cycle by which crypto OHLC are defined
    # end_date = datetime.now().replace(hour=20, minute=0, second=0, microsecond=0)
    # ^^^ ONLY WHEN RUNING LOCALLY, on github actions this will retrieve different data, so just use datetime.now() and schedule the run at 8pm EST = 0:00 UTC
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)

    url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': int(start_date.timestamp()),
        'to': int(end_date.timestamp())
    }
    
    response = safe_request(url, params)
    
    if response and response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['date', 'price'])
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['market_cap'] = pd.DataFrame(data['market_caps'])[1].values
        df['volume_24h'] = pd.DataFrame(data['total_volumes'])[1].values
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}" if response else "Failed to fetch data; no response.")
    
    # Include only the earliest day
    df['date'] = pd.to_datetime(df['date'])
    # Step 2: Extract the date part from the 'datetime' column
    df['no_hour_date'] = df['date'].dt.date
    earliest_date = df['no_hour_date'].min()
    filtered_df = df[df['no_hour_date'] == earliest_date]
    filtered_df = filtered_df.drop(columns='no_hour_date')
    
    return filtered_df

def safe_request(url, params, retries=20, backoff_factor=0.5):
    for i in range(retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                # We're being rate-limited; back off and retry
                sleep_time = backoff_factor * (2 ** i)
                print(f"Rate limit hit. Waiting {sleep_time:.2f} seconds before retrying...")
                time.sleep(sleep_time)
            else:
                # Other errors, break the retry loop and return None
                print(f"Request failed with status code {response.status_code}.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Retrying...")
            time.sleep(backoff_factor * (2 ** i))

    return None

In [42]:
eth_raw = fetch_recent_data("ethereum")
eth_raw

,date,price,market_cap,volume_24h
0,2024-05-14 02:11:34.254,2945.254691,3.536196e+11,1.390973e+10
1,2024-05-14 02:17:15.569,2943.702108,3.536196e+11,1.392130e+10
2,2024-05-14 02:22:44.720,2943.326684,3.536752e+11,1.394738e+10
3,2024-05-14 02:26:52.257,2944.395684,3.536752e+11,1.394404e+10
4,2024-05-14 02:31:42.672,2944.299962,3.535251e+11,1.390346e+10
...,...,...,...,...
257,2024-05-14 23:35:33.686,2881.806758,3.462601e+11,1.117981e+10
258,2024-05-14 23:41:13.076,2882.118003,3.461856e+11,1.112505e+10
259,2024-05-14 23:45:54.287,2881.638415,3.461856e+11,1.123470e+10
260,2024-05-14 23:51:07.643,2883.120296,3.463060e+11,1.125938e+10


### transform this data into ohlc (open, high, low, close) data for the past day

In [43]:
eth_new = eth_raw.resample('D', on='date').agg({
    'price': ['first', 'max', 'min', 'last', 'mean'],
    'volume_24h': 'last',
    'market_cap': 'last'
}).reset_index()
eth_new.columns = ['date', 'open', 'high', 'low', 'close', 'avg_price', 'volume', 'market_cap']
eth_new['volume_eth'] = eth_new['volume'] / eth_new['avg_price']
eth_new = eth_new[['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']]

eth_new


,date,open,high,low,close,volume,volume_eth,market_cap
0,2024-05-14,2945.254691,2952.313133,2867.874351,2881.795992,1.125873e+10,3.873530e+06,3.463060e+11


## Combine this recent data with what we already have in hopsworks feature store

In [49]:
eth_old = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)
eth_old

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.32s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3080,2024-05-10,3035.000000,3050.000000,2889.000000,2910.000000,1.060000e+10,3.557434e+06,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.000000,2935.000000,2894.000000,2911.000000,5.900000e+09,2.033385e+06,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3082,2024-05-12,2912.000000,2951.000000,2906.000000,2931.000000,5.200000e+09,1.779026e+06,3.581000e+11,1.0,0.010926,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3083,2024-05-13,2916.796334,2984.551884,2868.448720,2948.303455,1.393028e+10,4.735937e+06,3.541556e+11,0.0,-0.009737,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


In [50]:
eth_old = eth_old[:-1]
eth_old

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.891200,0.921500,0.87500,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.906200,0.944700,0.89200,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.924900,1.030000,0.90580,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.990000,1.010000,0.94050,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.988700,1.010000,0.93750,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,2024-05-09,2977.000000,3055.000000,2958.00000,3036.000000,9.600000e+09,3.190231e+06,3.670000e+11,0.0,-0.018445,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3080,2024-05-10,3035.000000,3050.000000,2889.00000,2910.000000,1.060000e+10,3.557434e+06,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.000000,2935.000000,2894.00000,2911.000000,5.900000e+09,2.033385e+06,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3082,2024-05-12,2912.000000,2951.000000,2906.00000,2931.000000,5.200000e+09,1.779026e+06,3.581000e+11,1.0,0.010926,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


### Combine the raw, untransformed portion of the old date with the new data we just got from CoinGecko

In [51]:
columns_needed = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']

# Convert date columns to datetime objects, in case they are not already
eth_old['date'] = pd.to_datetime(eth_old['date'])
eth_new['date'] = pd.to_datetime(eth_new['date'])

# Remove any timezone information to make the comparison straightforward
eth_old['date'] = eth_old['date'].dt.tz_localize(None)
eth_new['date'] = eth_new['date'].dt.tz_localize(None)

if not any(eth_new['date'].isin(eth_old['date'])):
    # Since eth_new's date isn't found, just concatenate it
    eth_combined = pd.concat([eth_old[columns_needed], eth_new], ignore_index=True)
else:
    # If the date is found, remove the last row from eth_old and add on the new row
    eth_combined = pd.concat([eth_old.iloc[:-1], eth_new], ignore_index=True)

eth_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,open,high,low,close,volume,volume_eth,market_cap
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07
...,...,...,...,...,...,...,...,...
3080,2024-05-10,3035.000000,3050.000000,2889.000000,2910.000000,1.060000e+10,3.557434e+06,3.650000e+11
3081,2024-05-11,2909.000000,2935.000000,2894.000000,2911.000000,5.900000e+09,2.033385e+06,3.569000e+11
3082,2024-05-12,2912.000000,2951.000000,2906.000000,2931.000000,5.200000e+09,1.779026e+06,3.581000e+11
3083,2024-05-13,2916.796334,2984.551884,2868.448720,2948.303455,1.393028e+10,4.735937e+06,3.541556e+11


### Use this combined raw dataset to regenerate all the response and predictor variables for the past observations, and for the new observation that was just added

In [46]:
def add_response_vars(data):
    df = data.copy()
    tmw_avg_high_close = (df["high"].shift(-1) + df["close"].shift(-1)) / 2
    df['tmw_percent_increase_to_avg_high_low'] = ((tmw_avg_high_close - df['close']) / df['close'])
    df['tmw_1_0_percent_increase_binary'] = (df['tmw_percent_increase_to_avg_high_low'] >= .01).astype(int)

    # Last row doesn't have a value for tomorrow's return, but it was assigned 0's for response variable. Convert them to NA
    df.loc[df.index[-1], ['tmw_percent_increase_to_avg_high_low', 'tmw_1_0_percent_increase_binary']] = pd.NA

    return df

def add_predictor_vars(data):
    df = data.copy()

    # define 'helper' columns
    perc_from_low_to_high = (df['high'] - df['low']) / df['low']

    tmw_vol = df["volume"].shift(-1)
    tmw_perc_change_vol = (tmw_vol - df['volume']) / df['volume']


    # Create binary columns for each month (1-12)
    month_names = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    for i in range(1, 13):  # Months are typically 1-12
        month_name = month_names[i-1]  # Get the month name from the list
        df[f'{month_name}'] = (df['date'].dt.month == i).astype(int)

    # Create binary columns for each day
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    for i, day in enumerate(days):
        df[f'{day.lower()}'] = (df['date'].dt.dayofweek == i).astype(int)

    horizons = [2,5,10,25,50,100] 
    for horizon in horizons:

        # Ratio - Close Price Simple Moving Average (SMA)
        sma_col = f"sma_{horizon}"
        df[sma_col] = df["close"] / ta.sma(df["close"], length=horizon)
        # Close Price Relative Strength Index (RSI)
        rsi_col = f"rsi_{horizon}"
        df[rsi_col] = ta.rsi(df["close"], length=horizon)

        # Ratio - Volume Simple Moving Average (SMA)
        sma_col = f"volume_sma_{horizon}"
        df[sma_col] = df["volume"] / ta.sma(df["volume"], length=horizon)
        # Volume Relative Strength Index (RSI)
        rsi_col = f"volume_rsi_{horizon}"
        df[rsi_col] = ta.rsi(df["volume"], length=horizon)

        # Ratio - High-Low Spread Simple Moving Average (SMA)
        sma_col = f"spread_sma_{horizon}"
        df[sma_col] = perc_from_low_to_high / ta.sma(perc_from_low_to_high, length=horizon)
        # High-Low Spread Relative Strength Index (RSI)
        rsi_col = f"spread_rsi_{horizon}"
        df[rsi_col] = ta.rsi(perc_from_low_to_high, length=horizon)

        # Sum of the number of days in the past horizon that had a certain percent change in price from a day's close to the next day's (high+close)/2
        df[f"last_{horizon}_day_40th_to_50th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.002684) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.008635)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_to_60th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.008635) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.016393)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_to_70th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.016393) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.026398)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_to_80th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.026398) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.041709)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_to_90th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.041709) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.070177)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_to_95th_pct_price_change_count"] = ((df['tmw_percent_increase_to_avg_high_low'] > 0.070177) & (df['tmw_percent_increase_to_avg_high_low'] <= 0.106177)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_40th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.002684).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.008635).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.016393).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.026398).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.041709).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.070177).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_95th_pct_price_change_count"] = (df['tmw_percent_increase_to_avg_high_low'] > 0.106177).astype(int).shift(1).rolling(horizon).sum()
                
        
        # Sum of the number of days in the past horizon that had a certain percent chamge in volume from one day to the next
        tmw_vol = df["volume"].shift(-1)
        tmw_perc_change_vol = (tmw_vol - df['volume']) / df['volume']
        df[f"last_{horizon}_day_40th_to_50th_pct_volume_change_count"] = ((tmw_perc_change_vol > -0.069444) & (tmw_perc_change_vol <= -0.008197)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_to_60th_pct_volume_change_count"] = ((tmw_perc_change_vol > -0.008197) & (tmw_perc_change_vol <= 0.061069)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_to_70th_pct_volume_change_count"] = ((tmw_perc_change_vol > 0.061069) & (tmw_perc_change_vol <= 0.137500)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_to_80th_pct_volume_change_count"] = ((tmw_perc_change_vol > 0.137500) & (tmw_perc_change_vol <= 0.276316)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_to_90th_pct_volume_change_count"] = ((tmw_perc_change_vol > 0.276316) & (tmw_perc_change_vol <= 0.544118)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_to_95th_pct_volume_change_count"] = ((tmw_perc_change_vol > 0.544118) & (tmw_perc_change_vol <= 0.885965)).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_40th_pct_volume_change_count"] = (tmw_perc_change_vol > -0.069444).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_pct_volume_change_count"] = (tmw_perc_change_vol > -0.008197).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_pct_volume_change_count"] = (tmw_perc_change_vol > 0.061069).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_pct_volume_change_count"] = (tmw_perc_change_vol > 0.137500).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_pct_volume_change_count"] = (tmw_perc_change_vol > 0.276316).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_pct_volume_change_count"] = (tmw_perc_change_vol > 0.544118).astype(int).shift(1).rolling(horizon).sum()
        df[f"last_{horizon}_day_95th_pct_volume_change_count"] = (tmw_perc_change_vol > 0.885965).astype(int).shift(1).rolling(horizon).sum()

        # Sum of the number of days in the past horizon that had a certain percent chamge in the difference between low and high price
        prc_from_low_to_high = (df['high'] - df['low']) / df['low']
        df[f"last_{horizon}_day_40th_to_50th_pct_spread_count"] = ((prc_from_low_to_high > 0.043674) & (prc_from_low_to_high <= 0.053125)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_to_60th_pct_spread_count"] = ((prc_from_low_to_high > 0.053125) & (prc_from_low_to_high <= 0.064163)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_to_70th_pct_spread_count"] = ((prc_from_low_to_high > 0.064163) & (prc_from_low_to_high <= 0.077524)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_to_80th_pct_spread_count"] = ((prc_from_low_to_high > 0.077524) & (prc_from_low_to_high <= 0.097901)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_to_90th_pct_spread_count"] = ((prc_from_low_to_high > 0.097901) & (prc_from_low_to_high <= 0.137338)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_to_95th_pct_spread_count"] = ((prc_from_low_to_high > 0.137338) & (prc_from_low_to_high <= 0.191936)).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_40th_pct_spread_count"] = (prc_from_low_to_high > 0.043674).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_50th_pct_spread_count"] = (prc_from_low_to_high > 0.053125).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_60th_pct_spread_count"] = (prc_from_low_to_high > 0.064163).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_70th_pct_spread_count"] = (prc_from_low_to_high > 0.077524).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_80th_pct_spread_count"] = (prc_from_low_to_high > 0.097901).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_90th_pct_spread_count"] = (prc_from_low_to_high > 0.137338).astype(int).rolling(horizon).sum()
        df[f"last_{horizon}_day_95th_pct_spread_count"] = (prc_from_low_to_high > 0.191936).astype(int).rolling(horizon).sum()

    # only keep the 96 predictor set, the one we found to be the best
    predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count', 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']
    cols_for_hopsworks = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']
    df = df[cols_for_hopsworks + ['tmw_1_0_percent_increase_binary', 'tmw_percent_increase_to_avg_high_low'] + predictors_96]

    return df

In [52]:
eth_responses = add_response_vars(eth_combined)
eth_predictors_and_responses = add_predictor_vars(eth_responses)
eth_new_days_transformed = eth_predictors_and_responses.iloc[-2:]
eth_new_days_transformed

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
3083,2024-05-13,2916.796334,2984.551884,2868.448720,2948.303455,1.393028e+10,4.735937e+06,3.541556e+11,0.0,-0.010599,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3084,2024-05-14,2945.254691,2952.313133,2867.874351,2881.795992,1.125873e+10,3.873530e+06,3.463060e+11,NaN,NaN,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


In [53]:
# Replace rows in eth_old with rows from eth_new_days_transformed based on the 'date' column
eth_old2 = eth_old.copy()
eth_old2.loc[eth_old2['date'].isin(eth_new_days_transformed['date'])] = eth_new_days_transformed.loc[eth_new_days_transformed['date'].isin(eth_old2['date'])]

# Find rows in eth_new_days_transformed that are not in eth_old
unique_dates_mask = ~eth_new_days_transformed['date'].isin(eth_old2['date'])
eth_new_unique = eth_new_days_transformed[unique_dates_mask]

# Concatenate the updated eth_old with the unique new rows
eth_transformed = pd.concat([eth_old, eth_new_unique], ignore_index=True)

eth_transformed

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3080,2024-05-10,3035.000000,3050.000000,2889.000000,2910.000000,1.060000e+10,3.557434e+06,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.000000,2935.000000,2894.000000,2911.000000,5.900000e+09,2.033385e+06,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3082,2024-05-12,2912.000000,2951.000000,2906.000000,2931.000000,5.200000e+09,1.779026e+06,3.581000e+11,1.0,0.010926,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3083,2024-05-13,2916.796334,2984.551884,2868.448720,2948.303455,1.393028e+10,4.735937e+06,3.541556e+11,0.0,-0.009737,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


## Save the data back into Hopsworks

In [34]:
eth_transformed['tmw_1_0_percent_increase_binary'] = eth_transformed['tmw_1_0_percent_increase_binary'].astype('Int64')

upload_data(eth_transformed, 'eth_ohlc_transformed', 2)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 3085/3085 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: eth_ohlc_transformed_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/711829/jobs/named/eth_ohlc_transformed_2_offline_fg_materialization/executions
